# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_5 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, 0] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi | PI |
| --- | --- | --- | --- |--- | --- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 | 0 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 | 0 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 | 0 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 | 0 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 | 0 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 | 0 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | 0 | 25 |

# Setup

In [21]:
import galois
import numpy as np
from utils import (
    generator1,
    generator2,
    curve_order,
    normalize,
    validate_point,
    GPoint,
    SRS,
    numbers_to_hash,
    patch_galois,
    dump_proof,
    dump_circuit
)

G1 = generator1()
G2 = generator2()

patch_galois(galois.Poly)

In [22]:
# initial values for x and y
x = 2
y = 3

# to switch between encrypted (ECC) and unencrypted mode (prime field p=241)
encrypted = False

# Prime field p
p = 241 if not encrypted else curve_order
# p = 241
Fp = galois.GF(p)

# 2x^2 - x^2y^2 + 3
out = 2 * x**2 - x**2 * y**2 + 3
print(f"out = {out}")

# We have 7 gates, next power of 2 is 8
n = 7
n = 2 ** int(np.ceil(np.log2(n)))
assert n & n - 1 == 0, "n must be a power of 2"

# Find primitive root of unity
omega = Fp.primitive_root_of_unity(n)
assert omega ** (n) == 1, f"omega (ω) {omega} is not a root of unity"

roots = Fp([omega**i for i in range(n)])
print(f"roots = {roots}")

out = -25
roots = [  1  30 177   8 240 211  64 233]


## Witness & Gates

In [23]:
def pad_array(a, n):
    return a + [0] * (n - len(a))


# witness vectors
a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]
pi = [0, 0, 0, 0, 0, 0, 25]

# gate vectors
ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, 0]

# pad vectors to length n
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
pi = pad_array(pi, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)

print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")
print(f"ql = {ql}")
print(f"qr = {qr}")
print(f"qm = {qm}")
print(f"qc = {qc}")
print(f"qo = {qo}")
print(f"pi = {pi}")

a = [2, 2, 3, 4, 4, 8, -28, 0]
b = [2, 2, 3, 0, 9, 36, 3, 0]
c = [4, 4, 9, 8, 36, -28, -25, 0]
ql = [0, 0, 0, 2, 0, 1, 1, 0]
qr = [0, 0, 0, 0, 0, -1, 0, 0]
qm = [1, 1, 1, 1, 1, 0, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3, 0]
qo = [-1, -1, -1, -1, -1, -1, 0, 0]
pi = [0, 0, 0, 0, 0, 0, 25, 0]


## Permutations

In [24]:
def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(
            f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}"
        )

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(
            f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}"
        )


ai = range(0, n)
bi = range(n, 2 * n)
ci = range(2 * n, 3 * n)

sigma = {
    ai[0]: ai[0],
    ai[1]: ai[1],
    ai[2]: ai[2],
    ai[3]: ci[0],
    ai[4]: ci[1],
    ai[5]: ci[3],
    ai[6]: ci[5],
    ai[7]: ai[7],
    bi[0]: bi[0],
    bi[1]: bi[1],
    bi[2]: bi[2],
    bi[3]: bi[3],
    bi[4]: ci[2],
    bi[5]: ci[4],
    bi[6]: bi[6],
    bi[7]: bi[7],
    ci[0]: ai[3],
    ci[1]: ai[4],
    ci[2]: bi[4],
    ci[3]: ai[5],
    ci[4]: bi[5],
    ci[5]: ai[6],
    ci[6]: ci[6],
    ci[7]: ci[7],
}

k1 = 2
k2 = 4
c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for r in c_roots:
    assert not int(r) in check, f"Duplicate root {r} in {c_roots}"
    check.add(int(r))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, c_roots)

print("\n\n--- Cosest ---")
print(f"c0 = {c1_roots}")
print(f"c1 = {c2_roots}")
print(f"c2 = {c3_roots}")

print("\n\n--- Sigma ---")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")
print(f"sigma3 = {sigma3}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     30 |     30
a2 |      3 |    177 |    177
a3 |      4 |      8 |      4
a4 |      4 |    240 |    120
a5 |      8 |    211 |     32
a6 |    -28 |     64 |    121
a7 |      0 |    233 |    233
-- | --     | --     | --    
b0 |      2 |      2 |      2
b1 |      2 |     60 |     60
b2 |      3 |    113 |    113
b3 |      0 |     16 |     16
b4 |      9 |    239 |    226
b5 |     36 |    181 |    237
b6 |      3 |    128 |    128
b7 |      0 |    225 |    225
-- | --     | --     | --    
c0 |      4 |      4 |      8
c1 |      4 |    120 |    240
c2 |      9 |    226 |    239
c3 |      8 |     32 |    211
c4 |     36 |    237 |    181
c5 |    -28 |    121 |     64
c6 |    -25 |     15 |     15
c7 |      0 |    209 |    209


--- Cosest ---
c0 = [  1  30 177   8 240 211  64 233]
c1 = [  2  60 113  16 239 181 128 225]
c2 = [  4 120 226  32 237 121  15 209]


--- Sigma ---
sigma1 = [  1  30 177   4 120  32 121 

## Gate Polynomials

In [25]:
def to_galois_array(vector, field):
    # normalize to positive values
    a = [x % field.order for x in vector]
    return field(a)


def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)


QL = to_poly(roots, ql, Fp)
QR = to_poly(roots, qr, Fp)
QM = to_poly(roots, qm, Fp)
QC = to_poly(roots, qc, Fp)
QO = to_poly(roots, qo, Fp)
PI = to_poly(roots, pi, Fp)

print("--- Gate Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")
print(f"PI = {PI}")

--- Gate Polynomials ---
QL = 187x^7 + 38x^6 + 119x^5 + 60x^4 + 70x^3 + 22x^2 + 106x + 121
QR = 64x^7 + 8x^6 + x^5 + 211x^4 + 177x^3 + 233x^2 + 240x + 30
QM = 57x^7 + 211x^6 + 73x^5 + 211x^4 + 168x^3 + 211x^2 + 184x + 91
QC = 24x^7 + 90x^6 + 217x^5 + 151x^4 + 24x^3 + 90x^2 + 217x + 151
QO = 7x^7 + 38x^6 + 169x^5 + 9x^3 + 22x^2 + 56x + 180
PI = 200x^7 + 27x^6 + 41x^5 + 214x^4 + 200x^3 + 27x^2 + 41x + 214


## Permutation Polynomials

In [26]:
S1 = to_poly(roots, sigma1, Fp)
S2 = to_poly(roots, sigma2, Fp)
S3 = to_poly(roots, sigma3, Fp)

I1 = to_poly(roots, c1_roots, Fp)
I2 = to_poly(roots, c2_roots, Fp)
I3 = to_poly(roots, c3_roots, Fp)

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"  I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"  I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"  I3({roots[i]:{padding}}) = {I3(roots[i]):{padding}} "
    s += f"  S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"  S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    s += f"  S3({roots[i]:{padding}}) = {S3(roots[i]):{padding}} "
    print(s)

    assert I1(roots[i]) == roots[i], f"I1({roots[i]}) != {roots[i]}"
    assert I2(roots[i]) == k1 * roots[i], f"I2({roots[i]}) != {k1 * roots[i]}"
    assert I3(roots[i]) == k2 * roots[i], f"I3({roots[i]}) != {k2 * roots[i]}"

    assert S1(roots[i]) == sigma1[i], f"S1({roots[i]}) != {sigma1[i]}"
    assert S2(roots[i]) == sigma2[i], f"S2({roots[i]}) != {sigma2[i]}"
    assert S3(roots[i]) == sigma3[i], f"S3({roots[i]}) != {sigma3[i]}"

: 

In [ ]:
def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=field)


Zh = to_vanishing_poly(roots, Fp)
for x in roots:
    assert Zh(x) == 0

print("--- Vanishing Polynomial ---")
print(f"Zh = {Zh}")

--- Vanishing Polynomial ---
Zh = x^8 + 240


## CRS Construction

In [ ]:
def generate_tau(encrypted=False):
    return SRS(Fp.Random(), n) if encrypted else Fp.Random()


tau = generate_tau(encrypted=encrypted)
print(f"--- Tau ---")
print(tau)

--- Tau ---
204


# Prover

## Round 1

In [ ]:
random_b = [Fp.Random() for i in range(0, 9)]

bA = galois.Poly(random_b[:2], field=Fp)
bB = galois.Poly(random_b[2:4], field=Fp)
bC = galois.Poly(random_b[4:6], field=Fp)

_A = to_poly(roots, a, Fp)
_B = to_poly(roots, b, Fp)
_C = to_poly(roots, c, Fp)

A = _A + bA * Zh
B = _B + bB * Zh
C = _C + bC * Zh

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + c(x)*qo(x) + qpi(x) + qc(x)
G = A * QL + B * QR + A * B * QM + C * QO + QC + PI

print("--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(
        f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}"
    )
    assert G(roots[i]) == 0, f"G({roots[i]}) != 0"

assert G % Zh == 0, f"G(x) % Zh(x) != 0"

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    print(s)

round1 = [A(tau), B(tau), C(tau)]
print("\n\n--- Round 1 ---")
print(f"Round 1 = {round1}")

--- Gate Constraints Polynomial ---
G = 125x^25 + 3x^24 + 108x^23 + 229x^22 + 141x^21 + 201x^20 + 111x^18 + 2x^17 + 36x^16 + 167x^15 + 182x^14 + 238x^13 + 58x^12 + 238x^11 + x^10 + 166x^9 + 126x^8 + 207x^7 + 71x^6 + 103x^5 + 223x^4 + 3x^3 + 129x^2 + 189x + 76
gate #0 G(1) = 0 --> OK
gate #1 G(30) = 0 --> OK
gate #2 G(177) = 0 --> OK
gate #3 G(8) = 0 --> OK
gate #4 G(240) = 0 --> OK
gate #5 G(211) = 0 --> OK
gate #6 G(64) = 0 --> OK
gate #7 G(233) = 0 --> OK
i =   0 -->   1    A(  1) =   2    B(  1) =   2    C(  1) =   4 
i =   1 -->  30    A( 30) =   2    B( 30) =   2    C( 30) =   4 
i =   2 --> 177    A(177) =   3    B(177) =   3    C(177) =   9 
i =   3 -->   8    A(  8) =   4    B(  8) =   0    C(  8) =   8 
i =   4 --> 240    A(240) =   4    B(240) =   9    C(240) =  36 
i =   5 --> 211    A(211) =   8    B(211) =  36    C(211) = 213 
i =   6 -->  64    A( 64) = 213    B( 64) =   3    C( 64) = 216 
i =   7 --> 233    A(233) =   0    B(233) =   0    C(233) =   0 


--- Round 1 ---


## Round 2

In [ ]:
beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)

_F = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
_G = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

acc_eval = [Fp(1)]
for i in range(0, n):
    acc_eval.append(acc_eval[-1] * (_F(roots[i]) / _G(roots[i])))
assert acc_eval.pop() == Fp(1)
ACC = galois.lagrange_poly(roots, Fp(acc_eval))
print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

bZ = galois.Poly(random_b[6:9], field=Fp)
print(f"bZ = {bZ}")
Z = bZ * Zh + ACC

print("\n\n--- Z Polynomial ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

round2 = [Z(tau)]
print("\n\n--- Round 2 ---")
print(f"Round 2 = {round2}")



--- Accumulator Polynomial ---
ACC(x) = 65x^7 + 30x^6 + 61x^5 + 182x^4 + 11x^3 + 158x^2 + 211x + 6
bZ = 104x^2 + 235x + 214


--- Z Polynomial ---
Z(x) = 104x^10 + 235x^9 + 214x^8 + 65x^7 + 30x^6 + 61x^5 + 182x^4 + 11x^3 + 54x^2 + 217x + 33
Z(1) = 1
Z(30) = 38
Z(177) = 229
Z(8) = 125
Z(240) = 28
Z(211) = 96
Z(64) = 12
Z(233) = 1


--- Round 2 ---
Round 2 = [GF(92, order=241)]


# Round 3

In [ ]:
def shift_poly(poly: galois.Poly, omega: Fp):
    coeffs = poly.coeffs[::-1]
    coeffs = [c * omega**i for i, c in enumerate(coeffs)]
    return galois.Poly(coeffs[::-1], field=poly.field)


alpha = numbers_to_hash(round1 + round2, Fp)

Zomega = shift_poly(Z, omega=omega)
for r in roots:
    print(f"Z({r:3}) = {Z(r):3} -> Zω({r:3}) = {Zomega(r):3}")

print("\n\n--- Zω Polynomial ---")
print(f"Z(ωx) = {Zomega}")

L1 = galois.lagrange_poly(roots, Fp([1] + [Fp(0)] * (n - 1)))

print("\n\n--- L1 Polynomial ---")
print(f"L1(x) = {L1}")
for i, r in enumerate(roots):
    print(f"L1({r}) = {L1(r)}")
    assert L1(r) == (Fp(1) if i == 0 else Fp(0))

T0 = G
assert T0 % Zh == 0, f"T0(x) % Zh(x) != 0"

T1 = (_F * Z - _G * Zomega) * alpha
assert T1 % Zh == 0, f"T1(x) % Zh(x) != 0"

T2 = (Z - galois.Poly([1], field=Fp)) * L1 * alpha**2
assert T2 % Zh == 0, f"T2(x) % Zh(x) != 0"

T = T0 + T1 + T2
assert T % Zh == 0, f"T(x) % Zh(x) != 0"

for r in roots:
    assert T(r) == 0, f"T({r}) != 0"

T = T // Zh

print("\n\n--- T Polynomial ---")
print(f"T(x) = {T}")

t_coeffs = T.coeffs[::-1]

Tl = galois.Poly(t_coeffs[:n][::-1], field=Fp)
Tm = galois.Poly(t_coeffs[n : 2 * (n)][::-1], field=Fp)
Th = galois.Poly(t_coeffs[2 * (n) :][::-1], field=Fp)

X_n = galois.Poly.Degrees([n, 0], coeffs=[1, 0], field=Fp)
X_2n = galois.Poly.Degrees([2 * (n), 0], coeffs=[1, 0], field=Fp)
# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)
assert T.degree == 3 * n + 5

b10 = Fp.Random()
b11 = Fp.Random()

Tl = Tl + b10 * X_n
Tm = Tm - b10 + b11 * X_n
Th = Th - b11
assert T == (Tl + X_n * Tm + X_2n * Th)

print("\n\n--- T' ---")
print(f"Tl(x) = {Tl}")
print(f"Tm(x) = {Tm}")
print(f"Th(x) = {Th}")

round3 = [Tl(tau), Tm(tau), Th(tau)]
print("\n\n--- Round 3 ---")
print(f"Round 3 = {round3}")

Z(  1) =   1 -> Zω(  1) =  38
Z( 30) =  38 -> Zω( 30) = 229
Z(177) = 229 -> Zω(177) = 125
Z(  8) = 125 -> Zω(  8) =  28
Z(240) =  28 -> Zω(240) =  96
Z(211) =  96 -> Zω(211) =  12
Z( 64) =  12 -> Zω( 64) =   1
Z(233) =   1 -> Zω(233) =   1


--- Zω Polynomial ---
Z(ωx) = 92x^10 + 61x^9 + 214x^8 + 203x^7 + 233x^6 + 98x^5 + 59x^4 + 88x^3 + 159x^2 + 3x + 33


--- L1 Polynomial ---
L1(x) = 211x^7 + 211x^6 + 211x^5 + 211x^4 + 211x^3 + 211x^2 + 211x + 211
L1(1) = 1
L1(30) = 0
L1(177) = 0
L1(8) = 0
L1(240) = 0
L1(211) = 0
L1(64) = 0
L1(233) = 0


--- T Polynomial ---
T(x) = 69x^29 + 71x^28 + 13x^27 + 87x^26 + 134x^25 + 4x^24 + 220x^23 + 224x^22 + 98x^21 + 218x^20 + 106x^19 + 44x^18 + 204x^17 + 139x^16 + 101x^15 + 137x^14 + 111x^13 + 146x^12 + 97x^11 + 40x^10 + 144x^9 + 105x^8 + 78x^7 + 140x^6 + 132x^5 + 134x^4 + 49x^3 + 87x^2 + 200x + 38


--- T' ---
Tl(x) = 89x^8 + 78x^7 + 140x^6 + 132x^5 + 134x^4 + 49x^3 + 87x^2 + 200x + 38
Tm(x) = 10x^8 + 101x^7 + 137x^6 + 111x^5 + 146x^4 + 97x^3 + 40x^2 +

## Round 4

In [ ]:
zeta = numbers_to_hash(round1 + round2 + round3, Fp)

a_zeta = A(zeta)
b_zeta = B(zeta)
c_zeta = C(zeta)
s1_zeta = S1(zeta)
s2_zeta = S2(zeta)
z_omega_zeta = Zomega(zeta)

round4 = [a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta]
print("\n\n--- Round 4 ---")
print(f"Round 4 = {round4}")



--- Round 4 ---
Round 4 = [GF(56, order=241), GF(174, order=241), GF(57, order=241), GF(140, order=241), GF(217, order=241), GF(224, order=241)]


# Round 5

In [ ]:
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)

pi_zeta = PI(zeta)

R = QM * a_zeta * b_zeta + QL * a_zeta + QR * b_zeta + QO * c_zeta + QC + pi_zeta
R += (
    Z
    * (a_zeta + beta * zeta + gamma)
    * (b_zeta + beta * zeta * k1 + gamma)
    * (c_zeta + beta * zeta * k2 + gamma)
    * alpha
)
R -= (
    z_omega_zeta
    * (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * (c_zeta + beta * S3 + gamma)
    * alpha
)
R += (Z - Fp(1)) * L1(zeta) * alpha**2
R -= Zh(zeta) * (Tl + zeta**n * Tm + zeta ** (2 * n) * Th)

print("\n\n--- R ---")
print(f"R(x) = {R}")

X_minus_zeta = galois.Poly([1, -zeta], field=Fp)
print(f"X - zeta = {X_minus_zeta}")

Wzeta = (
    R
    + (A - a_zeta) * v
    + (B - b_zeta) * v**2
    + (C - c_zeta) * v**3
    + (S1 - s1_zeta) * v**4
    + (S2 - s2_zeta) * v**5
)

assert Wzeta % X_minus_zeta == 0, f"Wzeta(x) % X - zeta != 0"
Wzeta = Wzeta // X_minus_zeta

X_minus_omega_zeta = galois.Poly([1, -(omega * zeta)], field=Fp)
print(f"X - ω*zeta = {X_minus_omega_zeta}")

Womega_zeta = Z - z_omega_zeta
assert Womega_zeta % X_minus_omega_zeta == 0, f"Womega_zeta(x) % X - ω*zeta != 0"
Womega_zeta = Womega_zeta // X_minus_omega_zeta

round5 = [Wzeta(tau), Womega_zeta(tau)]
print("\n\n--- Round 5 ---")
print(f"Round 5 = {round5}")

u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)



--- R ---
R(x) = 27x^13 + 164x^12 + 47x^11 + 162x^10 + 38x^9 + 180x^8 + 214x^7 + 142x^6 + 196x^5 + 218x^4 + 180x^3 + 236x^2 + 16x + 177
X - zeta = x + 93
X - ω*zeta = x + 139


--- Round 5 ---
Round 5 = [GF(1, order=241), GF(183, order=241)]


In [ ]:
proof = {
    "A": round1[0],
    "B": round1[1],
    "C": round1[2],
    "Z": round2[0],
    "Tl": round3[0],
    "Tm": round3[1],
    "Th": round3[2],
    "Wzeta": round5[0],
    "Womega_zeta": round5[1],
    "a_zeta": round4[0],
    "b_zeta": round4[1],
    "c_zeta": round4[2],
    "s1_zeta": round4[3],
    "s2_zeta": round4[4],
    "z_omega_zeta": round4[5],
    "Fp": p,
}

dump_proof(proof, "proof.json")

circuit = {
    "QM": QM.coeffs,
    "QL": QL.coeffs,
    "QR": QR.coeffs,
    "QO": QO.coeffs,
    "QC": QC.coeffs,
    "PI": PI.coeffs,
    "Zh": Zh.coeffs,
    "L1": L1.coeffs,
    "S1": S1.coeffs,
    "S2": S2.coeffs,
    "S3": S3.coeffs,
    "k1": k1,
    "k2": k2,
    "tau": tau.tau if encrypted else tau,
    "Fp": p,
    "omega": omega,
    "n": n,
    "encrypted": encrypted,
}

dump_circuit(circuit, "circuit.json")

# Verifier

In [ ]:
# These evaluations are calculated beforehand during the setup phase
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)

# Values provided by the prover (round 1 to 5) is a proof.
a_exp = round1[0]
b_exp = round1[1]
c_exp = round1[2]

z_exp = round2[0]

tl_exp = round3[0]
tm_exp = round3[1]
th_exp = round3[2]

# Note: verifier has to verify that the following values are in the correct Fp field
a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, z_omega_zeta = round4

w_zeta_exp = round5[0]
w_omega_zeta_exp = round5[1]

# Note: verifier has to verify that the following values are on the curve
if encrypted:
    validate_point(qm_exp)
    validate_point(ql_exp)
    validate_point(qr_exp)
    validate_point(qo_exp)
    validate_point(qc_exp)
    validate_point(z_exp)
    validate_point(s1_exp)
    validate_point(s2_exp)
    validate_point(s3_exp)
    validate_point(tl_exp)
    validate_point(tm_exp)
    validate_point(th_exp)
    validate_point(a_exp)
    validate_point(b_exp)
    validate_point(c_exp)
    validate_point(w_zeta_exp)
    validate_point(w_omega_zeta_exp)

beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)
alpha = numbers_to_hash(round1 + round2, Fp)
zeta = numbers_to_hash(round1 + round2 + round3, Fp)
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)
u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)

In [ ]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)
PI_z = PI(zeta)

r0 = (
    PI_z
    - L1_z * alpha**2
    - (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * (c_zeta + gamma)
    * z_omega_zeta
    * alpha
)

In [ ]:
D_exp = (
    qm_exp * a_zeta * b_zeta
    + ql_exp * a_zeta
    + qr_exp * b_zeta
    + qo_exp * c_zeta
    + qc_exp
)

D_exp += z_exp * (
    (a_zeta + beta * zeta + gamma)
    * (b_zeta + beta * zeta * k1 + gamma)
    * (c_zeta + beta * zeta * k2 + gamma)
    * alpha
    + L1_z * alpha**2
    + u
)

D_exp -= (
    s3_exp
    * (a_zeta + beta * s1_zeta + gamma)
    * (b_zeta + beta * s2_zeta + gamma)
    * alpha
    * beta
    * z_omega_zeta
)

D_exp -= (tl_exp + tm_exp * zeta**n + th_exp * zeta ** (2 * n)) * Zh_z

In [ ]:
F_exp = (
    D_exp
    + a_exp * v
    + b_exp * v**2
    + c_exp * v**3
    + s1_exp * v**4
    + s2_exp * v**5
)

print(f"F_exp = {F_exp}")

F_exp = 192


In [ ]:
E_exp = (
    -r0
    + v * a_zeta
    + v**2 * b_zeta
    + v**3 * c_zeta
    + v**4 * s1_zeta
    + v**5 * s2_zeta
    + u * z_omega_zeta
)

if encrypted:
    E_exp = G1 * E_exp

print(f"E_exp = {E_exp}")

E_exp = 59


In [ ]:
e1 = w_zeta_exp + w_omega_zeta_exp * u
e2 = (
    w_zeta_exp * zeta
    + w_omega_zeta_exp * (u * zeta * omega)
    + F_exp
    + (E_exp * Fp(p - 1))
)

if encrypted:
    pairing1 = tau.tau2.pair(e1)
    pairing2 = G2.pair(e2)

    print(f"pairing1 = {pairing1}")
    print(f"pairing2 = {pairing2}")

    assert pairing1 == pairing2, f"pairing1 != pairing2"
else:
    print("\n\n--- e1, e2 ---")
    print(f"e1 = {e1 * tau} = {e1} * tau")
    print(f"e2 = {e2}")
    assert e1 * tau == e2



--- e1, e2 ---
e1 = 117 = 75 * tau
e2 = 117
